**Exercise 1:**

In [0]:
def get_train_test_split(dataset_full,ratio):
  train_set=[]
  test_set=[]
  size_dataset_full=len(dataset_full)
  size_test=int(round(size_dataset_full*ratio,0))
  list_test_indices=random.sample(range(size_dataset_full), size_test)
  for i,example in enumerate(dataset_full):
    if i in list_test_indices: test_set.append(example)
    else: train_set.append(example)
  return train_set,test_set

# To verify
train_set_, test_set_=get_train_test_split(dataset_full,0.2)
print ("Size training set: "+str(len(train_set_)))
print ("Size test set: "+str(len(test_set_)))

**Exercise 2:**

In [0]:
list_num_features=[100,500,1000]
best_f1_dev=0.0
for num_features in list_num_features:
  vocabulary=get_vocabulary(new_train_set, num_features)  
  svm_clf=train_svm_classifier(new_train_set, vocabulary)
  X_dev=[]
  for instance in new_dev_set:
    vector_instance=get_vector_text(vocabulary,instance[0])
    X_dev.append(vector_instance)
  X_dev=np.asarray(X_dev)
  Y_dev_predictions=svm_clf.predict(X_dev)
  f1_dev=f1_score(Y_dev_gold, Y_dev_predictions, average='macro')
  print ("F1-Score with "+str(num_features)+": "+str(round(f1_dev,3)))
  if f1_dev>=best_f1_dev:
    best_f1_dev=f1_dev
    best_num_features=num_features
    best_vocabulary=vocabulary
    best_svm_clf=svm_clf
print ("\nBest F-Score overall in the dev set is "+str(round(best_f1_dev,3))+" with "+str(best_num_features)+" features.")
# Now we test the best classifier (in the dev set) on the test set
X_test=[]
Y_test=[]
for instance in new_test_set:
  vector_instance=get_vector_text(best_vocabulary,instance[0])
  X_test.append(vector_instance)
  Y_test.append(instance[1])
Y_test_gold=np.asarray(Y_test)
best_X_test=np.asarray(X_test)
best_Y_text_predictions=best_svm_clf.predict(best_X_test)
print("\nPerformance in the test set\n")
print(classification_report(Y_test_gold, best_Y_text_predictions))

**Exercise 3:**

In [0]:
num_folds=3
num_features=1000
kf = KFold(n_splits=num_folds)
random.shuffle(dataset_full)
kf.get_n_splits(dataset_full)
j_fold=0
accuracy_total=0.0
for train_index, test_index in kf.split(dataset_full):
  j_fold+=1
  train_set_fold=[]
  test_set_fold=[]
  for i,instance in enumerate(dataset_full):
    if i in train_index:
      train_set_fold.append(instance)
    else:
      test_set_fold.append(instance)
  vocabulary_fold=get_vocabulary(train_set_fold, num_features)
  svm_clf_fold=train_svm_classifier(train_set_fold, vocabulary_fold)
  X_test_fold=[]
  Y_test_fold=[]
  for instance in test_set_fold:
    vector_instance=get_vector_text(vocabulary_fold,instance[0])
    X_test_fold.append(vector_instance)
    Y_test_fold.append(instance[1])
  Y_test_fold_gold=np.asarray(Y_test_fold)
  X_test_fold=np.asarray(X_test_fold)
  Y_test_predictions_fold=svm_clf_fold.predict(X_test_fold)
  accuracy_fold=accuracy_score(Y_test_fold_gold, Y_test_predictions_fold)
  accuracy_total+=accuracy_fold
  print ("Fold "+str(j_fold)+"/"+str(num_folds)+" completed. Accuracy: "+str(accuracy_fold))
  
average_accuracy=accuracy_total/num_folds
print ("\nAverage Accuracy: "+str(round(average_accuracy,3)))